In [15]:
# !pip install -r ./requirements.txt

In [1]:
# !pip uninstall virny -y

Found existing installation: virny 0.3.0
Uninstalling virny-0.3.0:
  Successfully uninstalled virny-0.3.0


In [3]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_for_uncertainty_experiments

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [6]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [7]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import StudentPerformancePortugueseDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning2

from source.experiment_interface import run_exp_iter_with_eq_odds

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [8]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'EqOdds_student_performance'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIR_INTERVENTION_PARAMS_LST = [True]
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     'EqOddsPostprocessing', EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                'EqOddsPostprocessing', EXPERIMENT_NAME, 'student_performance_por_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [9]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [10]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [11]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'b35e31f2-bb03-494f-a24c-b9264e383667',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  b35e31f2-bb03-494f-a24c-b9264e383667


## Initialize custom objects

In [12]:
data_loader = StudentPerformancePortugueseDataset()
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,18,4,4,2,2,0,4,3,4,1,...,course,mother,yes,no,no,no,yes,yes,no,no
1,17,1,1,1,2,0,5,3,3,1,...,course,father,no,yes,no,no,no,yes,yes,no
2,15,1,1,1,2,0,4,3,2,2,...,other,mother,yes,no,no,no,yes,yes,yes,no
3,15,4,2,1,3,0,3,2,2,1,...,home,mother,no,yes,no,yes,yes,yes,yes,yes
4,16,3,3,1,2,0,4,3,2,1,...,home,father,no,yes,no,no,yes,yes,no,no


In [13]:
data_loader.X_data.shape

(649, 32)

## Run experiment iterations

### Experiment iteration 1

In [14]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [15]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [16]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          with_tuning=True,
                          # with_tuning=False,
                          # tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name='StudentPerformancePortugueseDataset',
                          verbose=True)

2024-01-04 06:14:22 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 100,
 'session_uuid': 'b35e31f2-bb03-494f-a24c-b9264e383667'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 06:14:22 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_options:  True
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 

2024-01-04 06:14:30 experiment_interface.py INFO    : Models are tuned and saved to a file
INFO:root:Models are tuned and saved to a file


Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [17]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240104__111430.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'EqOddsPostprocessing', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [18]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name='StudentPerformancePortugueseDataset',
                          verbose=True)

2024-01-04 06:24:28 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 200,
 'session_uuid': 'b35e31f2-bb03-494f-a24c-b9264e383667'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 06:24:28 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 06:24:28 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_options:  True
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [19]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240104__111430.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'EqOddsPostprocessing', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [20]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name='StudentPerformancePortugueseDataset',
                          verbose=True)

2024-01-04 06:34:06 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 300,
 'session_uuid': 'b35e31f2-bb03-494f-a24c-b9264e383667'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 06:34:06 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 06:34:06 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_options:  True
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [21]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240104__111430.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'EqOddsPostprocessing', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name='StudentPerformancePortugueseDataset',
                          verbose=True)

2024-01-04 06:43:39 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 400,
 'session_uuid': 'b35e31f2-bb03-494f-a24c-b9264e383667'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 06:43:39 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 06:43:39 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_options:  True
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
       'cat__romantic_yes', 

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240104__111430.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'EqOddsPostprocessing', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name='StudentPerformancePortugueseDataset',
                          verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Student_Performance_Por_20240104__111430.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                      'EqOddsPostprocessing', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning2(experiment_seed)

In [ ]:
run_exp_iter_with_eq_odds(data_loader=exp_iter_data_loader,
                          experiment_seed=experiment_seed,
                          test_set_fraction=TEST_SET_FRACTION,
                          db_writer_func=db_writer_func,
                          fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                          models_params_for_tuning=models_params_for_tuning,
                          metrics_computation_config=metrics_computation_config,
                          custom_table_fields_dct=custom_table_fields_dct,
                          # with_tuning=True,
                          with_tuning=False,
                          tuned_params_df_paths=tuned_params_df_paths,
                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                          dataset_name='StudentPerformancePortugueseDataset',
                          verbose=True)